In [8]:
import pandas as pd
#아래 함수들은 저번에 만든 함수들을 그대로 가져왔다.

In [70]:
def classify_gu(df,a):
  idx1 = df[df['시군구'] != a].index
  gu_df = df.drop(idx1)
  return gu_df 

In [71]:
def classify_fast(df):
  f_df = df[df['타입'] != 'AC완속']
  idxs = f_df.index
  s_df = df.drop(idxs)
  return f_df, s_df

In [72]:
# 함수 수정
def classify_time(df):
  f1 = df[df['이용가능시간'] == '24시간 이용가능' ]
  f2 = df[df['이용가능시간'] == '주증/주말 : 24시간' ]
  f3 = df[df['이용가능시간'] == '24시간' ]
  f4 = df[df['이용가능시간'] == '~' ]
  f5 = df[df['이용가능시간'] == '00:00 ~ 23:59' ]
  f6 = df[df['이용가능시간'] == 'NaN' ]
  f7 = df[df['이용가능시간'] == '주중/주말 : 24시간']
  f8 = df[df['이용가능시간'] == '24시간(단, 22시이후근무자운영' ]
   
  #이용 가능 시간 column에 시간 말고 다른 제한조건이 같이 있는 raw도 있습니다.
  c1 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자외출입제한' ]
  c2 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자 외 출입제' ]
  c3 = df[df['이용가능시간'] == '입주민 전용 24시간' ]
  c4 = df[df['이용가능시간'] == '24시간 이용가능(외부인 미개방)' ]
 
  full = pd.concat([f1,f2,f3,f4,f5,f6,f7,f8])
  temporary_c = pd.concat([c1, c2, c3,c4])
  fullfc = pd.concat([full, temporary_c])
  idx = fullfc.index
  nonfull = df.drop(idx)
  return full, nonfull, temporary_c

In [73]:
def classify_constraints(df,temporary_c):
  c1 = df[df['비고'] == '입주민만 사용가능 거주자외 출입제한' ]
  c2 = df[df['비고'] == '입주민만 사용가능 거주자 외 출입제한' ]
  c3 = df[df['비고'] == '입주민 등 거주자 외 출입 제한' ]
  c4 = df[df['비고'] == '공동주택 입주민 전용' ]
  c5 = df[df['비고'] == '거주자외 출입제한' ]
  c6 = df[df['비고'] == '입주민으로 사용제한' ]
  

  c = pd.concat([temporary_c,c1,c2,c3,c4,c5,c6])
  nonc1 = df.drop(c1.index)
  nonc2 = nonc1.drop(c2.index)
  return c,nonc2

In [74]:
def classify_all(df,a):
  dfgu = classify_gu(df,a)
  # fast slow classify
  fast, slow = classify_fast(dfgu)
  # time classify
  fast_full , fast_nonfull , temporary_fast_cons = classify_time(fast) 
  slow_full , slow_nonfull , temporary_slow_cons = classify_time(slow)
  # constraints classify
  ffc, ffnc = classify_constraints(fast_full,temporary_fast_cons)
  nfc, nfnc = classify_constraints(fast_nonfull,temporary_fast_cons)
  fsc, fsnc = classify_constraints(slow_full, temporary_slow_cons)
  nsc, nsnc = classify_constraints(slow_nonfull, temporary_slow_cons)
  return ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc


In [75]:
df = pd.read_excel('e_dongdaemoon_df.xlsx')
a = '동대문구 '

In [76]:
ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc = classify_all(df,a)


In [104]:
# 기존에 DataFrame을 만들때 raw에 우리가 원하는 동이 다 들어가지 않는 문제가 생겼다.
# 이를 해결하기위해 전체 데이터셋df에서 행정동들을 set으로 처리해줬다.
# 그럼 중복된 동은 하나로 저장되어 그 구에 있는 행정동을 다 받을 수있다.
# 이것을 다시 list로 만들고 DataFame에 index로 사용하겠다.
a = df['Unnamed: 13.1'].values
index = set(a)

In [78]:
gu_df = pd.DataFrame(index = index)

gu_df['ffnc'] =  ffnc['Unnamed: 13.1'].value_counts()
gu_df['ffc'] =  ffc['Unnamed: 13.1'].value_counts()
gu_df['nfnc'] =  nfnc['Unnamed: 13.1'].value_counts()
gu_df['nfc'] =  nfc['Unnamed: 13.1'].value_counts()
gu_df['fsnc'] =  fsnc['Unnamed: 13.1'].value_counts()
gu_df['fsc'] =  fsc['Unnamed: 13.1'].value_counts()
gu_df['nsnc'] =  nsnc['Unnamed: 13.1'].value_counts()
gu_df['nsc'] =  nsc['Unnamed: 13.1'].value_counts()


In [80]:
#동대문구 완료!
gu_df.to_excel('ff_dong_df.xlsx')
gu_df.head(14)

,ffnc,ffc,nfnc,nfc,fsnc,fsc,nsnc,nsc
답십리2동,2.0,NaN,1.0,NaN,7.0,NaN,7.0,NaN
휘경1동,NaN,NaN,NaN,NaN,10.0,NaN,23.0,NaN
이문2동,NaN,NaN,1.0,NaN,15.0,5.0,2.0,5.0
이문1동,NaN,NaN,1.0,NaN,42.0,NaN,1.0,NaN
휘경2동,4.0,NaN,3.0,NaN,10.0,4.0,6.0,4.0
용신동,1.0,NaN,NaN,NaN,NaN,3.0,4.0,3.0
전농2동,1.0,NaN,2.0,NaN,25.0,8.0,1.0,8.0
제기동,2.0,NaN,NaN,NaN,6.0,10.0,2.0,10.0
답십리1동,2.0,NaN,1.0,NaN,36.0,NaN,NaN,NaN
회기동,2.0,NaN,2.0,NaN,1.0,5.0,NaN,5.0


In [81]:
df = pd.read_excel('e_dongjack_df.xlsx')
a = '동작구 '

In [82]:
ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc = classify_all(df,a)


In [83]:
a = df['Unnamed: 13.1'].values
index = set(a)

In [84]:
gu_df = pd.DataFrame(index = index)

gu_df['ffnc'] =  ffnc['Unnamed: 13.1'].value_counts()
gu_df['ffc'] =  ffc['Unnamed: 13.1'].value_counts()
gu_df['nfnc'] =  nfnc['Unnamed: 13.1'].value_counts()
gu_df['nfc'] =  nfc['Unnamed: 13.1'].value_counts()
gu_df['fsnc'] =  fsnc['Unnamed: 13.1'].value_counts()
gu_df['fsc'] =  fsc['Unnamed: 13.1'].value_counts()
gu_df['nsnc'] =  nsnc['Unnamed: 13.1'].value_counts()
gu_df['nsc'] =  nsc['Unnamed: 13.1'].value_counts()

In [85]:
#동작구 완료!
gu_df.to_excel('ff_dongjack_df.xlsx')
gu_df.head(14)

,ffnc,ffc,nfnc,nfc,fsnc,fsc,nsnc,nsc
노량진2동,1.0,NaN,1.0,NaN,10.0,NaN,NaN,NaN
노량진1동,NaN,NaN,NaN,NaN,28.0,2.0,NaN,2.0
사당5동,NaN,NaN,NaN,NaN,9.0,NaN,3.0,NaN
양평2동,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN
상도4동,NaN,NaN,1.0,NaN,NaN,1.0,5.0,1.0
사당2동,2.0,NaN,NaN,NaN,1.0,2.0,NaN,2.0
사당1동,NaN,NaN,1.0,NaN,11.0,NaN,6.0,NaN
신대방2동,NaN,NaN,1.0,NaN,15.0,3.0,9.0,NaN
상도1동,1.0,NaN,1.0,NaN,35.0,20.0,2.0,20.0
상도3동,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
df = pd.read_excel('e_mapo_df.xlsx')
a = '마포구 '

In [87]:
ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc = classify_all(df,a)


In [88]:
a = df['Unnamed: 13.1'].values
index = set(a)

In [89]:
gu_df = pd.DataFrame(index = index)

gu_df['ffnc'] =  ffnc['Unnamed: 13.1'].value_counts()
gu_df['ffc'] =  ffc['Unnamed: 13.1'].value_counts()
gu_df['nfnc'] =  nfnc['Unnamed: 13.1'].value_counts()
gu_df['nfc'] =  nfc['Unnamed: 13.1'].value_counts()
gu_df['fsnc'] =  fsnc['Unnamed: 13.1'].value_counts()
gu_df['fsc'] =  fsc['Unnamed: 13.1'].value_counts()
gu_df['nsnc'] =  nsnc['Unnamed: 13.1'].value_counts()
gu_df['nsc'] =  nsc['Unnamed: 13.1'].value_counts()

In [90]:
#마포구 완료!
gu_df.to_excel('ff_mapo_df.xlsx')
gu_df.head(14)

,ffnc,ffc,nfnc,nfc,fsnc,fsc,nsnc,nsc
성산2동,11.0,NaN,1.0,NaN,22.0,NaN,7.0,NaN
중동,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN
도화동,1.0,NaN,1.0,NaN,6.0,10.0,1.0,10.0
성산1동,1.0,NaN,2.0,NaN,NaN,NaN,1.0,NaN
망원2동,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN
상암동,8.0,NaN,3.0,NaN,54.0,18.0,35.0,20.0
아현동,1.0,NaN,3.0,NaN,14.0,10.0,35.0,16.0
합정동,3.0,NaN,NaN,NaN,1.0,NaN,8.0,NaN
용강동,7.0,NaN,8.0,NaN,21.0,NaN,24.0,NaN
서강동,1.0,NaN,2.0,NaN,19.0,6.0,4.0,6.0


In [91]:
df = pd.read_excel('e_seocho_df.xlsx')
a = '서초구 '

In [92]:
ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc = classify_all(df,a)


In [93]:
a = df['Unnamed: 13.1'].values
index = set(a)

In [94]:
gu_df = pd.DataFrame(index = index)

gu_df['ffnc'] =  ffnc['Unnamed: 13.1'].value_counts()
gu_df['ffc'] =  ffc['Unnamed: 13.1'].value_counts()
gu_df['nfnc'] =  nfnc['Unnamed: 13.1'].value_counts()
gu_df['nfc'] =  nfc['Unnamed: 13.1'].value_counts()
gu_df['fsnc'] =  fsnc['Unnamed: 13.1'].value_counts()
gu_df['fsc'] =  fsc['Unnamed: 13.1'].value_counts()
gu_df['nsnc'] =  nsnc['Unnamed: 13.1'].value_counts()
gu_df['nsc'] =  nsc['Unnamed: 13.1'].value_counts()

In [95]:
#서초구 완료!
gu_df.to_excel('ff_seocho_df.xlsx')
gu_df.head(14)

,ffnc,ffc,nfnc,nfc,fsnc,fsc,nsnc,nsc
반포본동,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0
방배4동,NaN,NaN,NaN,NaN,24.0,2.0,17.0,2.0
반포2동,2.0,NaN,NaN,NaN,20.0,NaN,19.0,NaN
방배2동,2.0,NaN,NaN,NaN,4.0,NaN,14.0,NaN
방배본동,3.0,NaN,NaN,NaN,16.0,NaN,8.0,NaN
서초4동,13.0,NaN,NaN,NaN,28.0,22.0,11.0,22.0
내곡동,5.0,NaN,2.0,NaN,24.0,6.0,5.0,6.0
방배3동,NaN,NaN,NaN,NaN,17.0,1.0,9.0,NaN
잠원동,1.0,NaN,NaN,NaN,10.0,18.0,3.0,18.0
반포1동,6.0,NaN,1.0,NaN,69.0,57.0,7.0,54.0


In [96]:
df = pd.read_excel('e_seodaemoon_df.xlsx')
a = '서대문구 '

In [97]:
ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc = classify_all(df,a)


In [98]:
a = df['Unnamed: 13.1'].values
index = set(a)

In [99]:
gu_df = pd.DataFrame(index = index)

gu_df['ffnc'] =  ffnc['Unnamed: 13.1'].value_counts()
gu_df['ffc'] =  ffc['Unnamed: 13.1'].value_counts()
gu_df['nfnc'] =  nfnc['Unnamed: 13.1'].value_counts()
gu_df['nfc'] =  nfc['Unnamed: 13.1'].value_counts()
gu_df['fsnc'] =  fsnc['Unnamed: 13.1'].value_counts()
gu_df['fsc'] =  fsc['Unnamed: 13.1'].value_counts()
gu_df['nsnc'] =  nsnc['Unnamed: 13.1'].value_counts()
gu_df['nsc'] =  nsc['Unnamed: 13.1'].value_counts()

In [100]:
#서대문구 완료!
gu_df.to_excel('ff_seodaemoon_df.xlsx')
gu_df.head(14)

,ffnc,ffc,nfnc,nfc,fsnc,fsc,nsnc,nsc
충현동,1.0,NaN,NaN,NaN,3.0,NaN,4.0,NaN
천연동,1.0,NaN,NaN,NaN,9.0,3.0,2.0,3.0
홍제3동,NaN,NaN,NaN,NaN,5.0,NaN,2.0,1.0
신촌동,NaN,NaN,3.0,NaN,13.0,NaN,11.0,NaN
북아현동,NaN,NaN,NaN,NaN,2.0,10.0,NaN,10.0
홍은2동,1.0,NaN,NaN,NaN,11.0,2.0,2.0,2.0
연희동,1.0,NaN,3.0,NaN,12.0,NaN,4.0,NaN
홍은1동,1.0,NaN,NaN,NaN,5.0,5.0,1.0,5.0
북가좌1동,1.0,NaN,NaN,NaN,11.0,NaN,3.0,1.0
남가좌1동,NaN,NaN,NaN,NaN,NaN,56.0,2.0,56.0


In [101]:
a = pd.read_excel('ff_dong_df.xlsx')
b = pd.read_excel('ff_dongjack_df.xlsx')
c = pd.read_excel('ff_mapo_df.xlsx')
d = pd.read_excel('ff_seocho_df.xlsx')
e = pd.read_excel('ff_seodaemoon_df.xlsx')

In [102]:
final_df = pd.concat([a,b,c,d,e])

In [103]:
final_df.to_excel('final5gu_df.xlsx')
final_df.head()

,Unnamed: 0,ffnc,ffc,nfnc,nfc,fsnc,fsc,nsnc,nsc
0,답십리2동,2.0,NaN,1.0,NaN,7.0,NaN,7.0,NaN
1,휘경1동,NaN,NaN,NaN,NaN,10.0,NaN,23.0,NaN
2,이문2동,NaN,NaN,1.0,NaN,15.0,5.0,2.0,5.0
3,이문1동,NaN,NaN,1.0,NaN,42.0,NaN,1.0,NaN
4,휘경2동,4.0,NaN,3.0,NaN,10.0,4.0,6.0,4.0
